In [1]:
!pip install denoising_diffusion_pytorch

Defaulting to user installation because normal site-packages is not writeable
Looking in indexes: https://pypi.org/simple, https://pypi.ngc.nvidia.com

[notice] A new release of pip is available: 24.0 -> 24.3.1
[notice] To update, run: python -m pip install --upgrade pip


In [2]:
from PIL import Image
import shutil
import os
import random
from torchvision import transforms, datasets
from torch.utils.data import DataLoader
from pathlib import Path

def prepare_sysu_shape_dataset(
    dataset_path,
    train_path,
    eval_path,
    image_size=128,
    eval_ratio=0.2,
    batch_size=32
):
    """
    Prepare and preprocess the SYSU-Shape dataset for DDPM training, 
    saving resized images.
    
    Args:
        dataset_path (str): Path to the root of the SYSU-Shape dataset.
        train_path (str): Path to store the processed training data.
        eval_path (str): Path to store the processed evaluation data.
        image_size (int): Target size for resizing images.
        eval_ratio (float): Proportion of images to use for evaluation.
        batch_size (int): Batch size for DataLoader.
    
    Returns:
        DataLoader: DataLoader for training and evaluation datasets.
    """
    categories = ['car', 'boat', 'motorbike', 'airplane', 'bicycle']
    
    # Clear existing directories to avoid appending
    if os.path.exists(train_path):
        shutil.rmtree(train_path)  # Delete train_path directory
    if os.path.exists(eval_path):
        shutil.rmtree(eval_path)  # Delete eval_path directory
    
    os.makedirs(train_path, exist_ok=True)
    os.makedirs(eval_path, exist_ok=True)

    # separate the process of resize and toTensor and normalize, 
    # therefore, we let the images stored also being resized
    transform = transforms.Compose([
        transforms.Resize((image_size, image_size))  # Resize to fixed size
    ])
    
    for category in categories:
        image_dir = os.path.join(dataset_path, category, 'images')
        images = [f for f in os.listdir(image_dir) if f.endswith(('.jpg', '.png'))]

        random.shuffle(images)
        eval_size = int(len(images) * eval_ratio)

        # Split images
        eval_images = images[:eval_size]
        train_images = images[eval_size:]

        # Process and save resized images
        for img_set, output_dir in [(train_images, train_path), (eval_images, eval_path)]:
            category_path = os.path.join(output_dir, category)
            os.makedirs(category_path, exist_ok=True)
            
            for img in img_set:
                img_path = os.path.join(image_dir, img)
                with Image.open(img_path) as image:
                    resized_image = transform(image)  # Apply resizing transformation
                    resized_image.save(os.path.join(category_path, img))  # Save resized image
    
    # Clean up unwanted folders in train and eval directories
    for folder in [train_path, eval_path]:
        for subdir in os.listdir(folder):
            if subdir not in categories:  # If folder is not in the fixed categories
                subdir_path = os.path.join(folder, subdir)
                if os.path.isdir(subdir_path):  # Ensure it's a directory
                    print(f"Removing unwanted folder: {subdir_path}")
                    shutil.rmtree(subdir_path)
                    
    # Define preprocessing transformations for DataLoader
    loader_transform = transforms.Compose([
        transforms.ToTensor(),                        # Convert to PyTorch tensor
        transforms.Normalize((0.5,), (0.5,))          # Normalize to [-1, 1]
    ])

    # Create DataLoaders for training and evaluation datasets
    train_dataset = datasets.ImageFolder(root=train_path, transform=loader_transform)
    eval_dataset = datasets.ImageFolder(root=eval_path, transform=loader_transform)

    train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
    eval_loader = DataLoader(eval_dataset, batch_size=batch_size, shuffle=False)

    return train_loader, eval_loader


In [3]:
# call the method
train_loader, eval_loader = prepare_sysu_shape_dataset(
    dataset_path="./sysu-shape-dataset",
    train_path="./processed-datasets/train_data",
    eval_path="./processed-datasets/eval_data",
    eval_ratio=0.2,
    image_size=128,
    batch_size=32
)

We have to move images of all categories (now in subdirs) into the root dir of eval_data, this is required by the pytorch-fid calculation

In [3]:
import os
import shutil

def flatten_directory(root_dir):
    """
    Moves all images from subdirectories of the root directory to the root directory.
    
    Args:
        root_dir (str): Path to the root directory to flatten.
    """
    for subdir in os.listdir(root_dir):
        subdir_path = os.path.join(root_dir, subdir)
        if os.path.isdir(subdir_path):
            for filename in os.listdir(subdir_path):
                file_path = os.path.join(subdir_path, filename)
                if os.path.isfile(file_path):
                    # Move the file to the root directory
                    shutil.move(file_path, os.path.join(root_dir, filename))
            # Remove the now-empty subdirectory
            os.rmdir(subdir_path)

# Example usage
eval_data_dir = "./processed-datasets/eval_data"
flatten_directory(eval_data_dir)
print(f"All images have been moved to the root of {eval_data_dir}.")

All images have been moved to the root of ./processed-datasets/eval_data.


normalize the evaluation images to [0, 1]

In [5]:
import os
from PIL import Image
import numpy as np

def normalize_images_to_unit_range(input_dir, output_dir):
    """
    Normalize images in the input directory to [0, 1] and save them in the output directory.

    Args:
        input_dir (str): Path to the directory containing the images to normalize.
        output_dir (str): Path to the directory where normalized images will be saved.
    """
    os.makedirs(output_dir, exist_ok=True)
    for root, _, files in os.walk(input_dir):
        for file in files:
            if file.lower().endswith(('.png', '.jpg', '.jpeg')):
                try:
                    input_path = os.path.join(root, file)
                    output_path = os.path.join(output_dir, file)

                    # Open and normalize image
                    img = Image.open(input_path).convert("RGB")
                    img = img.resize((128, 128))  # Ensure images are the same size
                    img = (np.asarray(img) / 255.0).clip(0, 1)  # Normalize to [0, 1]

                    # Save the normalized image
                    normalized_img = Image.fromarray((img * 255).astype("uint8"))
                    normalized_img.save(output_path)
                except Exception as e:
                    print(f"Error processing image {file}: {e}")

# Paths for the real images
eval_data_dir = "./processed-datasets/eval_data"
normalized_eval_data_dir = "./processed-datasets/normalized_eval_data"

normalize_images_to_unit_range(eval_data_dir, normalized_eval_data_dir)

print(f"Normalized images saved to {normalized_eval_data_dir}")

Normalized images saved to ./processed-datasets/normalized_eval_data


In [6]:
from PIL import Image
import numpy as np
import os

def check_image_preprocessing(directory):
    for root, _, files in os.walk(directory):
        for file in files:
            try:
                img = Image.open(os.path.join(root, file))
                img_array = np.asarray(img)  # Convert to numpy array
                print(f"{file}: min={img_array.min()}, max={img_array.max()}, shape={img_array.shape}")
            except Exception as e:
                print(f"Error processing {file}: {e}")

# Check real images
check_image_preprocessing("./processed-datasets/normalized_eval_data")

# Check generated images
check_image_preprocessing("/scratch/kyq5pg/MLIA_final/samples/model-70")

voc2012_2008_004838.jpg: min=0, max=255, shape=(128, 128, 3)
voc2012_2008_006762.jpg: min=0, max=255, shape=(128, 128, 3)
voc2012_2010_003947.jpg: min=0, max=255, shape=(128, 128, 3)
101_schooner_image_0018.jpg: min=45, max=209, shape=(128, 128, 3)
voc2012_2010_000418.jpg: min=13, max=255, shape=(128, 128, 3)
voc2007_003013.jpg: min=0, max=255, shape=(128, 128, 3)
voc2012_2010_003701.jpg: min=1, max=229, shape=(128, 128, 3)
101_airplanes_image_0031.jpg: min=0, max=255, shape=(128, 128, 3)
voc2012_2009_000737.jpg: min=0, max=255, shape=(128, 128, 3)
voc2012_2009_001750.jpg: min=0, max=250, shape=(128, 128, 3)
voc2012_2008_000399.jpg: min=0, max=255, shape=(128, 128, 3)
voc2012_2010_003856.jpg: min=0, max=255, shape=(128, 128, 3)
101_airplanes_image_0279.jpg: min=0, max=255, shape=(128, 128, 3)
voc2012_2009_004556.jpg: min=0, max=255, shape=(128, 128, 3)
101_airplanes_image_0266.jpg: min=0, max=255, shape=(128, 128, 3)
101_airplanes_image_0015.jpg: min=0, max=255, shape=(128, 128, 3)
101